# **C.O.L.L.A.B.**
<hr style="border:2px solid black">

## Goal: 
* 
* 
<hr style="border:2px solid black">

# Imports

<hr style="border:2px solid black">

# **Acquire**
Plan --> **Acquire** --> Prepare --> Explore --> Model --> Deliver

* 

<hr style="border:2px solid black">

# **Prepare**

**Prepare Actions:**
* Removed

<hr style="border:2px solid black">

# **Explore**
Plan --> Acquire --> Prepare --> **Explore** --> Model --> Deliver

<hr style="border:2px solid black">

# **Modeling**
Plan --> Acquire --> Prepare --> Explore --> **Model** --> Deliver


<hr style="border:2px solid black">

# **Comparing Models**


<hr style="border:2px solid black">

## **Best Model Evaluation on Test**

### <u>Modeling Summary:</u>

>* 

<hr style="border:2px solid black">

# **Conclusions**


### <u>Modeling:</u>

>**<u>:</u>**
>* 

### <u>Recommendations:</u>
>* 

### <u>Next Steps:</u>
>* 